<a href="https://colab.research.google.com/github/tranbaquang1708/GraduationThesis/blob/develop/rbf_2d_comparetime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Time comparation betwwen vectorized code and nonvectorized code
# Dataset: 2D

In [2]:
import sys
import os
mp = os.path.abspath(os.path.join('/content/drive/My Drive/GraduationThesis/gitrepo/GraduationThesis/'))
sys.path.append(mp)

In [3]:
import numpy as np
import timeit
from modules import Rbf

In [4]:
# Create dataset
# Import point data
filename = mp + "/dataset/2d/superior.vert"
dataset,normal_vectors = Rbf.gen_from_txt2(filename, offset=0.01)

In [5]:
SETUP = """
import numpy as np
from numpy import ma
from scipy import linalg
from scipy.spatial.distance import squareform,pdist
import scipy
from __main__ import dataset
"""

In [6]:
# Non-vectorized version of solving weight function
LOOP_SOLVE = """
offset = 0.01
num_points = len(dataset)
num_on_points = int(num_points/3)

A = np.zeros((num_points, num_points))
for i in range(num_points):
  for j in range(num_points):
    norm = (dataset[i][0] - dataset[j][0])**2 + (dataset[i][1] - dataset[j][1])**2
    if norm != 0:
      A[i][j] = norm * np.log(np.sqrt(norm))

print("Data: " + str(num_on_points) + " points")
print("Memory usage: " + str(sys.getsizeof(A)) + " bytes")
print()

## b: the vector of the function values at the point from the data-set
b = np.zeros(num_points)

for i in range(num_on_points, 2*num_on_points):
  b[i] = -offset

for i in range(2*num_on_points,3*num_on_points):
  b[i] = offset

w = scipy.linalg.solve(A,b)
"""

In [7]:
# Vectorized version of solving weight function 
VEC_SOLVE= """
offset = 0.01
num_points = len(dataset)
num_on_points = int(num_points/3)

r = squareform(pdist(dataset))
A = (r*r*ma.log(r)).filled(0)

## b: the vector of the function values at the point from the data-set
b = np.zeros(num_points)

b[num_on_points:2*num_on_points] = -offset
b[2*num_on_points:3*num_on_points] = offset

w = scipy.linalg.solve(A,b)
"""

In [8]:
# Time comparison
loop_time = timeit.timeit(LOOP_SOLVE, SETUP, number = 1)
print("Non-vectorized: " + str(loop_time) + " second(s)")

vec_time = timeit.timeit(setup = SETUP, stmt = VEC_SOLVE, number = 1)
print("Vectorized: " + str(vec_time) + " second(s)")

print("Speedup: " + str(loop_time/vec_time) + " times")

Data: 518 points
Memory usage: 19319440 bytes

Non-vectorized: 14.009358008000163 second(s)
Vectorized: 0.30164727899955324 second(s)
Speedup: 46.442845612477456 times


In [9]:
# Solve weight function
w = Rbf.solve_w_func(dataset, offset=0.01, func=Rbf.thin_plate)

In [10]:
SETUP = """
import numpy as np
from numpy import ma
from scipy import linalg
from scipy.spatial.distance import squareform,pdist
import scipy
from __main__ import dataset,xx,yy, w
"""

In [11]:
# Non-vectorized version of sampling function
LOOP_SAMPLING = """
dimg = xx.shape
dimw = w.shape
z = np.zeros(dimg)
for i in range(dimg[0]):
  for j in range(dimg[1]):
    z[i][j] = 0.0
    for k in range(dimw[0]):
      nxy = (xx[i][j]-dataset[k][0])**2 + (yy[i][j]-dataset[k][1])**2
      if (nxy != 0):
        z[i][j] += w[k]*nxy*np.log(np.sqrt(nxy))
"""

In [12]:
# Vectorized version of sampling function
VEC_SAMPLING = """
dimg = xx.shape
dimw = w.shape
z = np.zeros(dimg)
for k in range(int(dimw[0])):
  r = (xx - dataset[k,0])**2 + (yy - dataset[k,1])**2
  z += w[k] * ((r*ma.log(np.sqrt(r))).filled(0))
"""

In [13]:
# Create 2D regular grid obtained
xx,yy = Rbf.grid(dataset[:,0], dataset[:,1])

In [14]:
# Time comparison
loop_time = timeit.timeit(LOOP_SAMPLING, SETUP, number=1)
print("Non-vectorized: " + str(loop_time) + " second(s)")

vec_time = timeit.timeit(VEC_SAMPLING, SETUP, number=1)
print("Vectorized: " + str(vec_time) + " second(s)")

print("Speedup: " + str(loop_time/vec_time) + " times")

Non-vectorized: 24.124633356999766 second(s)
Vectorized: 0.3050167789997431 second(s)
Speedup: 79.09280740591679 times
